## Phase 3 (Data Preparation): Cabin => Deck, KnownCabin

* Autorin: Anna (i3-Versicherung)
* Webseite: [Data Science Training - Kapitel 5](https://data-science.training/kapitel-5/)
* Datum: 23.03.2023

Zum nominalskalierten Attribut "Cabin" fehlen 1014 Werte. Wenn eine Kabinennummer angegeben ist, dann bezeichnet der erste Buchstabe das Deck, auf dem sich die Kabine befindet. Somit können wir ein neues Feature "Deck" bilden. Ähnlich wie beim Alter können wir auch zwei Kategorien unterschieden, nämlich ob die Kabine bekannt ist oder nicht. Dies führt zu einem weiteren Feature: "KnownCabin".

Zuerst werden wir das neue Feature "KnownCabin" bestimmen. Dann werden die fehlenden Werte des Attributs "Cabin" durch den festen Text "Unknown" ersetzt. Daraus wird dann später das U-Deck bzw. das Deck "U" abgeleitet.

In [4]:
# Pandas Paket (Package) importieren
#  Datenstrukturen und Datenanalyse, I/O
#  https://pandas.pydata.org/pandas-docs/stable/
import pandas as pd

In [5]:
# Trainings- und Testdaten als Pandas Data Frame (df) aus CSV-Dateien laden
#  (KNIME: "CSV Reader")
df_train = pd.read_csv('../../data/titanic/original/train.csv')
df_test  = pd.read_csv('../../data/titanic/original/test.csv')

In [6]:
# Trainings- und Testdaten zusammenführen
#  (KNIME "Concatenate")
df = pd.concat([df_train, df_test], ignore_index=True)

In [7]:
# Datentypen automatisch konvertieren
df = df.convert_dtypes()

In [8]:
# Spezielle Titanic-Statistik (Überlebenswahrscheinlichkeiten)
def dst_titanic_statistics(df, col):
    stats = df.groupby([col], as_index=False, observed=True)['PassengerId'].count() # Spalte "PassengerId" aus Titanic-Daten
    probs = df.groupby([col], as_index=False, observed=True)['Survived'].mean()     # Spalte "Survived"    aus Titanic-Daten
    stats = stats.merge(probs, on=col)
    stats = stats.rename(columns={'PassengerId': 'Anzahl', 'Survived': 'Überlebenswahrscheinlichkeit'})
    return stats

In [9]:
# Neues Feature "KnownCabin"
#  (KNIME: "Rule Engine")
df['KnownCabin'] = (df['Cabin'].notna()).astype('int')
# Statistik (Überlebenswahrscheinlichkeiten)
stats = dst_titanic_statistics(df, 'KnownCabin')
display(stats)

,KnownCabin,Anzahl,Überlebenswahrscheinlichkeit
0,0,1014,0.299854
1,1,295,0.666667


In [10]:
# Fehlende Werte
# Cabin (Nominalskala): 1014 fehlende Werte => Benutze den festen Wert 'Unknown'
df['Cabin'] = df['Cabin'].fillna('Unknown')

In [11]:
# Neues Feature "Deck"
#  (KNIME: "String Manipulation")
df['Deck'] = df['Cabin'].str[0]
# Statistik (Überlebenswahrscheinlichkeiten)
stats = dst_titanic_statistics(df, 'Deck')
display(stats)

,Deck,Anzahl,Überlebenswahrscheinlichkeit
0,A,22,0.466667
1,B,65,0.744681
2,C,94,0.59322
3,D,46,0.757576
4,E,41,0.75
5,F,21,0.615385
6,G,5,0.5
7,T,1,0.0
8,U,1014,0.299854


### Schlussfolgerungen

Das Deck T ist ein Ausreißer, denn es kommt nur in einem Datensatz vor.

Passagiere mit einem bekannten Deck haben eine höhere Überlebenschance als wenn das Deck unbekannt ist (U = Unknown). Motivation für ein neues (einfaches) Feature KnownCabin.

Insbesondere die Decks B, D und E führen zu einer hohen Überlebenswahrscheinlichkeit von ca. 75 %. Eine (logische) Erklärung hierfür gibt es jedoch nicht. Erwartet hätten wir eher, dass die Überlebenswahrscheinlichkeit von A  zu G abnimmt. Vielleicht ist die schlechte Datengrundlage dafür verantwortlich, denn die Mehrzahl der Werte (1014) fehlen. => besser nur "KnownCabin" als neues Feature benutzen.

In [13]:
display(df.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,KnownCabin,Deck
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,Unknown,S,0,U
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,Unknown,S,0,U
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S,1,C
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,Unknown,S,0,U
